In [308]:
import pandas as pd
import numpy as np
import os
import re
import json
import redis
from tqdm import tqdm

In [340]:
%load_ext autoreload
%autoreload 2
from load_grades import load_grades

[autoreload of load_grades failed: Traceback (most recent call last):
  File "/Users/brian/anaconda/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "/Users/brian/dev/projects/tcf/theCourseForum/grades/data/load_grades.py", line 6
    load_status = rds.execute_command('JSON.SET', 'grades', '.', grade_json)
                                                                           ^
IndentationError: unindent does not match any outer indentation level
]


Loading Grades_Fall_2015.xlsx
Loading Grades_Fall_2018.xlsx
Loading Grades_Fall_2014.xlsx
Loading Grades_Fall_2013.xlsx
Loading Grades_Spring_2016.xlsx
Loading Grades_Fall_2012.xlsx
Loading Grades_Spring_2015.xlsx
Loading Grades_Fall_2011.xlsx
Loading Grades_Spring_2014.xlsx
Loading Grades_Spring_2018.xlsx
Loading Grades_Spring_2013.xlsx
Loading Grades_Fall_2017.xlsx
Loading Grades_Fall_2016.xlsx
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of update_grades failed: Traceback (most recent call last):
  File "/Users/brian/anaconda/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
KeyboardInterrupt
]


ImportError: cannot import name 'load_grades'

In [338]:
def load_grades(data_dir, rds):
    grade_dict = validate_grade_files(data_dir)
    grade_df = merge_grades(grade_dict)
    grade_data = grades_dict(grade_df)
    return rds.execute_command('JSON.SET', 'grades', '.', json.dumps(grade_data))

In [339]:
rds = redis.StrictRedis(host='localhost', port=6379, db=0)
load_grades('excel_files', rds)

Loading Grades_Fall_2015.xlsx


/Users/brian/dev/projects/tcf/theCourseForum/grades/data/validate_grades.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  grade_df[grade_df.columns[25]] = grade_cols.sum(axis=1)


Loading Grades_Fall_2018.xlsx


/Users/brian/dev/projects/tcf/theCourseForum/grades/data/validate_grades.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return grade_df


Loading Grades_Fall_2014.xlsx
Loading Grades_Fall_2013.xlsx
Loading Grades_Spring_2016.xlsx
Loading Grades_Fall_2012.xlsx
Loading Grades_Spring_2015.xlsx
Loading Grades_Fall_2011.xlsx
Loading Grades_Spring_2014.xlsx
Loading Grades_Spring_2018.xlsx
Loading Grades_Spring_2013.xlsx
Loading Grades_Fall_2017.xlsx
Loading Grades_Fall_2016.xlsx
Loading Grades_Spring_2012.xlsx
Validating Grades_Fall_2015.xlsx
Validating Grades_Fall_2018.xlsx
Validating Grades_Fall_2014.xlsx
Validating Grades_Fall_2013.xlsx
Validating Grades_Spring_2016.xlsx
Validating Grades_Fall_2012.xlsx
Validating Grades_Spring_2015.xlsx
Validating Grades_Fall_2011.xlsx
Validating Grades_Spring_2014.xlsx
Validating Grades_Spring_2018.xlsx
Validating Grades_Spring_2013.xlsx
Validating Grades_Fall_2017.xlsx
Validating Grades_Fall_2016.xlsx
Validating Grades_Spring_2012.xlsx
SUCCESS: All numeric conversions in files validated.


...

 35%|███▌      | 1962/5573 [00:34<01:18, 45.94it/s]

 35%|███▌      | 1969/5573 [00:34<01:10, 50.87it/s]

 35%|███▌      | 1975/5573 [00:34<01:08, 52.30it/s]

 36%|███▌      | 1981/5573 [00:35<01:06, 54.14it/s]

 36%|███▌      | 1987/5573 [00:35<01:06, 53.84it/s]

 36%|███▌      | 1994/5573 [00:35<01:02, 57.39it/s]

 36%|███▌      | 2000/5573 [00:35<01:28, 40.21it/s]

 36%|███▌      | 2006/5573 [00:35<01:20, 44.44it/s]

 36%|███▌      | 2014/5573 [00:35<01:10, 50.13it/s]

 36%|███▌      | 2020/5573 [00:35<01:08, 51.57it/s]

 36%|███▋      | 2026/5573 [00:35<01:13, 48.53it/s]

 36%|███▋      | 2033/5573 [00:36<01:06, 53.29it/s]

 37%|███▋      | 2039/5573 [00:36<01:06, 53.25it/s]

 37%|███▋      | 2047/5573 [00:36<00:59, 58.84it/s]

 37%|███▋      | 2054/5573 [00:36<01:36, 36.51it/s]

 37%|███▋      | 2059/5573 [00:36<01:29, 39.18it/s]

 37%|███▋      | 2066/5573 [00:36<01:19, 44.33it/s]

 37%|███▋      | 2072/5573 [00:36<01:13, 47.45it/s]

 37%|███▋      | 2078/5573 [00:37<01:12, 48.34

 52%|█████▏    | 2923/5573 [00:54<00:50, 52.87it/s]

 53%|█████▎    | 2930/5573 [00:54<00:47, 55.95it/s]

 53%|█████▎    | 2937/5573 [00:54<00:46, 56.94it/s]

 53%|█████▎    | 2944/5573 [00:54<00:44, 59.22it/s]

 53%|█████▎    | 2951/5573 [00:54<00:42, 62.03it/s]

 53%|█████▎    | 2960/5573 [00:54<00:39, 66.60it/s]

 53%|█████▎    | 2967/5573 [00:55<00:39, 65.16it/s]

 53%|█████▎    | 2974/5573 [00:55<00:41, 63.22it/s]

 53%|█████▎    | 2981/5573 [00:55<00:54, 47.91it/s]

 54%|█████▎    | 2989/5573 [00:55<00:48, 53.44it/s]

 54%|█████▍    | 2997/5573 [00:55<00:44, 57.96it/s]

 54%|█████▍    | 3005/5573 [00:55<00:41, 62.33it/s]

 54%|█████▍    | 3012/5573 [00:55<00:40, 63.72it/s]

 54%|█████▍    | 3019/5573 [00:55<00:40, 62.89it/s]

 54%|█████▍    | 3026/5573 [00:56<00:40, 62.55it/s]

 54%|█████▍    | 3033/5573 [00:56<00:42, 60.17it/s]

 55%|█████▍    | 3040/5573 [00:56<00:41, 60.79it/s]

 55%|█████▍    | 3048/5573 [00:56<00:39, 63.92it/s]

 55%|█████▍    | 3056/5573 [00:56<00:37, 67.85

 70%|███████   | 3905/5573 [01:12<00:34, 48.28it/s]

 70%|███████   | 3912/5573 [01:12<00:31, 52.09it/s]

 70%|███████   | 3921/5573 [01:12<00:28, 57.38it/s]

 70%|███████   | 3928/5573 [01:12<00:28, 57.84it/s]

 71%|███████   | 3935/5573 [01:12<00:27, 60.16it/s]

 71%|███████   | 3942/5573 [01:12<00:27, 59.54it/s]

 71%|███████   | 3949/5573 [01:13<00:28, 56.87it/s]

 71%|███████   | 3955/5573 [01:13<00:28, 57.47it/s]

 71%|███████   | 3962/5573 [01:13<00:26, 60.70it/s]

 71%|███████   | 3969/5573 [01:13<00:26, 59.64it/s]

 71%|███████▏  | 3976/5573 [01:13<00:26, 59.45it/s]

 71%|███████▏  | 3984/5573 [01:13<00:24, 63.69it/s]

 72%|███████▏  | 3991/5573 [01:13<00:25, 62.62it/s]

 72%|███████▏  | 3998/5573 [01:13<00:25, 61.05it/s]

 72%|███████▏  | 4006/5573 [01:13<00:24, 63.72it/s]

 72%|███████▏  | 4013/5573 [01:14<00:25, 60.99it/s]

 72%|███████▏  | 4021/5573 [01:14<00:23, 65.08it/s]

 72%|███████▏  | 4028/5573 [01:14<00:36, 42.70it/s]

 72%|███████▏  | 4034/5573 [01:14<00:33, 46.25

 87%|████████▋ | 4870/5573 [01:29<00:11, 62.56it/s]

 88%|████████▊ | 4878/5573 [01:29<00:10, 64.36it/s]

 88%|████████▊ | 4885/5573 [01:29<00:10, 65.50it/s]

 88%|████████▊ | 4894/5573 [01:29<00:09, 69.42it/s]

 88%|████████▊ | 4902/5573 [01:29<00:10, 62.74it/s]

 88%|████████▊ | 4910/5573 [01:29<00:10, 65.82it/s]

 88%|████████▊ | 4918/5573 [01:29<00:09, 67.31it/s]

 88%|████████▊ | 4925/5573 [01:30<00:10, 62.98it/s]

 89%|████████▊ | 4933/5573 [01:30<00:10, 63.94it/s]

 89%|████████▊ | 4940/5573 [01:30<00:09, 63.40it/s]

 89%|████████▉ | 4947/5573 [01:30<00:09, 64.61it/s]

 89%|████████▉ | 4954/5573 [01:30<00:12, 49.82it/s]

 89%|████████▉ | 4962/5573 [01:30<00:10, 56.10it/s]

 89%|████████▉ | 4969/5573 [01:30<00:11, 53.58it/s]

 89%|████████▉ | 4976/5573 [01:31<00:11, 52.52it/s]

 89%|████████▉ | 4983/5573 [01:31<00:11, 52.67it/s]

 90%|████████▉ | 4991/5573 [01:31<00:10, 57.26it/s]

 90%|████████▉ | 4999/5573 [01:31<00:09, 61.18it/s]

 90%|████████▉ | 5006/5573 [01:31<00:09, 61.67

b'OK'

In [334]:
from IPython.display import display, HTML
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [322]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)

In [323]:
print(r.execute_command('JSON.SET', 'grades', '.', json.dumps(grade_data)))

b'OK'


In [324]:
print(json.loads(r.execute_command('JSON.GET', 'grades')))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [325]:
print(json.loads(r.execute_command('JSON.GET', 'grades', 'AAS')))

{'course_gpas': {'1010': 3.42, '1020': 3.64, '1559': 3.52, '2224': 3.52, '2559': 3.36, '2657': 3.94, '2700': 3.14, '3000': 2.94, '3200': 3.22, '3250': 3.55, '3280': 2.14, '3300': 3.08, '3456': 3.46, '3500': 3.38, '3559': 3.3, '3652': 3.59, '3749': 3.54, '3853': 3.63, '4500': 3.64, '4501': 3.43, '4570': 3.1, '5559': 3.78}, '1010': {'gpa': 3.42, 'distribution': {'A+': 21, 'A': 262, 'A-': 291, 'B+': 164, 'B': 144, 'B-': 70, 'C+': 31, 'C': 27, 'C-': 20, 'D+': 5, 'D': 2, 'D-': 0, 'F': 2, 'Other': 3, 'Drop': 331, 'W': 13, 'Total': 1386}, 'trends': {'2011Fall': 3.62, '2012Fall': 3.47, '2013Fall': 3.23, '2014Fall': 3.28, '2015Fall': 3.53, '2016Fall': 3.48, '2017Fall': 3.6, '2018Fall': 3.18}, 'professor_gpas': {'eko3q': 3.42, 'jdl2s': 3.25, 'lbl4a': 3.53, 'rf2n': 3.62, 'rtv4a': 3.47}, 'professors': {'eko3q': {'gpa': 3.42, 'distribution': {'A+': 0, 'A': 98, 'A-': 133, 'B+': 75, 'B': 63, 'B-': 37, 'C+': 14, 'C': 9, 'C-': 5, 'D+': 4, 'D': 0, 'D-': 0, 'F': 1, 'Other': 2, 'Drop': 180, 'W': 5, 'Total

In [328]:
pp.pprint(json.loads(r.execute_command('JSON.GET', 'grades', "CS['2110'].distribution")))

{   'A': 1495,
    'A+': 616,
    'A-': 793,
    'B': 508,
    'B+': 577,
    'B-': 228,
    'C': 97,
    'C+': 137,
    'C-': 46,
    'D': 29,
    'D+': 33,
    'D-': 10,
    'Drop': 803,
    'F': 54,
    'Other': 6,
    'Total': 5479,
    'W': 47}


In [329]:
pp.pprint(json.loads(r.execute_command('JSON.GET', 'grades', "CS['2110'].trends")))

{   '2011Fall': 3.285,
    '2012Fall': 3.3449999999999998,
    '2012Spring': 3.495,
    '2013Fall': 3.27,
    '2013Spring': 3.3899999999999997,
    '2014Fall': 3.563333333333334,
    '2014Spring': 3.65,
    '2015Fall': 3.766666666666667,
    '2015Spring': 3.745,
    '2016Fall': 3.5875,
    '2016Spring': 3.7,
    '2017Fall': 3.318,
    '2018Fall': 3.4325,
    '2018Spring': 3.2975}


In [330]:
pp.pprint(json.loads(r.execute_command('JSON.GET', 'grades', "CS['2110'].professor_gpas")))

{   'djs6d': 3.3,
    'dle7pm': 3.45,
    'eaw5cf': 3.53,
    'erg3wb': 3.18,
    'jh2jf': 3.42,
    'lat7h': 3.32,
    'nb3f': 3.59,
    'rp6zr': 3.32,
    'tbh3f': 3.44}


In [332]:
pp.pprint(json.loads(r.execute_command('JSON.GET', 'grades', "CS['2110']")))

{   'distribution': {   'A': 1495,
                        'A+': 616,
                        'A-': 793,
                        'B': 508,
                        'B+': 577,
                        'B-': 228,
                        'C': 97,
                        'C+': 137,
                        'C-': 46,
                        'D': 29,
                        'D+': 33,
                        'D-': 10,
                        'Drop': 803,
                        'F': 54,
                        'Other': 6,
                        'Total': 5479,
                        'W': 47},
    'gpa': 3.47,
    'professor_gpas': {   'djs6d': 3.3,
                          'dle7pm': 3.45,
                          'eaw5cf': 3.53,
                          'erg3wb': 3.18,
                          'jh2jf': 3.42,
                          'lat7h': 3.32,
                          'nb3f': 3.59,
                          'rp6zr': 3.32,
                          'tbh3f': 3.44},
    'professors': {  